### 4.1 jeju hotplace crawling

> url 함수 만들기

In [55]:
# 인스타그램의 검색 결과 url을 만드는 함수
def insta_searching(word):
    url = 'https://www.instagram.com/explore/tags/' + word
    return url

In [56]:
# selenium으로 접속 - 1
from selenium import webdriver

driver = webdriver.Chrome('./chromedriver')

In [57]:
# selenium으로 접속 - 2
import time

driver.get('https://www.instagram.com')
time.sleep(2)

* 인스타계정을 입력해서 자동 로그인하거나, 웹창에 직접 입력

In [ ]:
# selenium으로 접속 - 3
# 인스타그램 로그인
email = ''  # 인스타 계정
input_id = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[0]
input_id.clear()
input_id.send_keys(email)

password = ''  # 인스타 비밀번호
input_pw = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[1]
input_pw.clear()
input_pw.send_keys(password)
input_pw.submit()

In [58]:
# selenium으로 접속 - 4
word = '제주도맛집'
url = insta_searching(word)
driver.get(url)

> 게시글 정보 가져오기

In [63]:
# 첫 번째 게시글 찾아 클릭
import time

def select_first(driver):
    first = driver.find_element_by_css_selector('div._9AhH0')  # 태그명이 div, class명이 _9AhH0인 요소 찾기
    first.click()
    time.sleep(3)  # 3초 대기
    
select_first(driver)

In [64]:
# 게시글 정보 가져오기
import re
from bs4 import BeautifulSoup
import unicodedata

def get_content(driver):
    
    # 1. 현재 페이지의 HTML 정보 가져오기
    html = driver.page_source  # 현재 화면에 표시된 내용의 html 데이터를 가져올 수 있다.
    soup = BeautifulSoup(html, 'lxml')  # 이 정보들을 BS을 통해 필요한 정보들을 추출
    
    # 2. 본문 내용 가져오기
    try:
        content = soup.select('div.C4VMK > span')[0].text  
        # 태그명이 div, class명이 C4VMK인 태그 아래 span태그 모두 선택, 그 중에 첫 번째 태그 선택, 해당 태그의 텍스트 부분
        content = unicodedata.normalize('NFC', content)
        # Mac에서 작성된 경우 크롤링 데이터 엑셀파일로 저장시 한글 자음/모음이 분리되는 현상 발생
        # 저장하는 과정에서 unicodedata를 이용하여 'NFC' 방식으로(자음/모음 합쳐서 한글 처리하는 방식) 전환
    except:
        content = ' '  # 본문 내용을 작성하지 않으면 해당 부분을 찾을 수 없어 에러가 발생하기 때문에 빈 값 저장
    
    # 3. 본문 내용에서 해시태그 가져오기(정규표현식 활용)
    tags = re.findall(r'#[^\s#,\\]+', content)
    
    # 4. 작성일자 정보 가져오기
    date = soup.select('time._1o9PC.Nzb55')[0]['datetime'][:10]
    # 태그명이 time, class명이 _1o9PC,Nzb55인 태그 모두 선택, 그 중 첫 번째 태그 선택, 해당 태그의 datetime 속성값 중 앞에서부터 10자리
    
    # 5. 좋아요 수 가져오기
    try:
        like = soup.select('div.Nm9Fw > button')[0].text[4:-1]
        # 태그명이 div, class명이 Nm9Fw인 태그 아래 button 태그 모두 선택, 그 중에 첫 번째 태그 선택, 해당 태그의 텍스트 부분 중 5번째 글자~오른쪽 끝에서 두 번째까지
    except:
        like = 0
    
    # 6. 위치 정보 가져오기
    try:
        place = soup.select('div.M30cS')[0].text
        # 태그명이 div, class명이 M30cS인 태그 아래 첫 번째 태그의 텍스트 부분
        place = unicodedata.normalize('NFC', place)
    except:
        place = ' '
    
    # 7. 수집한 정보 저장하기
    data = [content, date, like, place, tags]
    return data

get_content(driver)

["(광고)지만 찐로컬맛집!!제주도민들의 흑돼지픽🎯'한라축산정육식당'🙏착한가격에 신선함까지☺만족스러운 식사로제주도에서 유명하고정육식당이라 내가👀눈으로 보고 골라서먹는 재미도 쏠쏠~🎼미리 연락시에 룸에서조용하게 먹을 수 있고넓은 실내에 주차공간까지제주시 흑돼지맛집은'한라축산정육식당'🔥⠀🧭한라축산정육식당🛫공항근처점:🗺제주 제주시 1100로 3128⏰매일 10:30~22:30 연중무휴📞064-745-5757⠀🧭애월,한림점🗺제주 제주시 한림읍 한림상로 84⏰매일 10:00 - 22:00📞064-796-3855#제주흑돼지맛집 #제주도흑돼지맛집#제주시맛집 #제주공항근처맛집#제주한림맛집 #제주맛집 #제주도맛집#애월맛집 #제주도애월맛집 #제주애월맛집",
 '2020-10-06',
 0,
 '',
 ['#제주흑돼지맛집',
  '#제주도흑돼지맛집',
  '#제주시맛집',
  '#제주공항근처맛집',
  '#제주한림맛집',
  '#제주맛집',
  '#제주도맛집',
  '#애월맛집',
  '#제주도애월맛집',
  '#제주애월맛집']]

In [65]:
# 다음 게시글 열기
def move_next(driver):
    right = driver.find_element_by_css_selector('a.coreSpriteRightPaginationArrow')
    right.click()
    time.sleep(3)
    
move_next(driver)

> 여러 게시글 정보 수집하기

1. 크롬 브라우저 열기
2. 인스타그램 검색페이지 url 만들기: insta_searching()
3. 검색페이지 접속하기
4. 첫 번째 게시물 클릭하기: select_first()
5. 비어있는 변수(results) 만들기
6. 게시글 정보 가져오기: get_content()
7. 게시글 정보 변수(results)에 추가하기
8. 다음 게시글 선택하기: move_next()    
:: 6-7-8 반복    
9. 게시글 정보 수집 완료(results)

In [67]:
# 인스타그램 크롤링
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import re

# 1. 크롬 브라우저 열기
driver = webdriver.Chrome('./chromedriver')

driver.get('https://www.instagram.com')
time.sleep(2)

* 인스타계정을 입력해서 자동 로그인하거나, 웹창에 직접 입력

In [ ]:
# 인스타그램 로그인
email = ''  # 인스타 계정
input_id = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[0]
input_id.clear()
input_id.send_keys(email)

password = ''  # 인스타 비밀번호
input_pw = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[1]
input_pw.clear()
input_pw.send_keys(password)
input_pw.submit()

In [74]:
# 2. 인스타그램 검색페이지 url 만들기: insta_searching()
word = '제주도여행'  # 원하는 검색어 입력
url = insta_searching(word)

# 3. 검색페이지 접속하기
driver.get(url)
time.sleep(3)

# 4. 첫 번째 게시물 클릭하기
select_first(driver)

# 5. 비어있는 변수(results) 만들기
results = [ ]

# 6,7,8. 여러 게시물 수집하기
target = 500  # 크롤링할 게시글 수 
for i in range(target):
    try:
        data = get_content(driver)  # 게시글 정보 가져오기
        results.append(data)
        move_next(driver)
    except:
        time.sleep(2)
        move_next(driver)
    # 게시글 수집에 오류 발생시(네트워크 문제 등의 이유로) 2초 대기 후 다음 게시글로 넘어가기

print(results[:2])

[['핑크뮬리 반 사람 반 🌷💗', '2020-10-06', '1,712', '카페 마노르블랑 제주', []], ['하이엔드만의 특제 마농소스가 듬뿍 들어간 제주마농빵과, 청귤이 듬~ 뿍 들어간 리얼청귤에이드로 선선한 가을 같이 즐겨요😚아침일찍 또는 저녁에 오시면 여유롭고 잔잔한 하이엔드를 느끼실수 있어요 갓구운 베이커리와 직접로스팅한 향긋한 커피로 하루를시작 또는 마무리 하세요✈️(1일1하이엔드, 1일 1마스크)#애월카페 #제주애월카페 #제주도카페 #제주카페 #제주시카페 #서귀포카페 #협재카페 #한림카페 #제주여행 #제주도여행 #제주맛집 #제주도맛집 #애월맛집 #제주도가볼만한곳 #제주기념선물#제주도효도여행 #제주가족여행 #제주로스터리카페 #하이엔드제주 #제주도연중무휴 #제주연중무휴 #애월하미 #그레이트제이 #하갈비국수 #애월당 #하복 #하이월드', '2020-10-07', '153', '하이엔드 제주', ['#애월카페', '#제주애월카페', '#제주도카페', '#제주카페', '#제주시카페', '#서귀포카페', '#협재카페', '#한림카페', '#제주여행', '#제주도여행', '#제주맛집', '#제주도맛집', '#애월맛집', '#제주도가볼만한곳', '#제주기념선물', '#제주도효도여행', '#제주가족여행', '#제주로스터리카페', '#하이엔드제주', '#제주도연중무휴', '#제주연중무휴', '#애월하미', '#그레이트제이', '#하갈비국수', '#애월당', '#하복', '#하이월드']]]


> 수집 데이터 저장

In [75]:
# 크롤링 결과 저장
import pandas as pd

results_df = pd.DataFrame(results)
results_df.columns = ['content', 'date', 'like', 'place', 'tags']
results_df.to_excel('./data/jejudo_yeohaeng.xlsx') 
# 제주맛집, 제주도맛집, 제주여행, 제주도여행 4가지 검색 후 저장

> 여러 엑셀 파일의 중복을 제거한 후 통합 저장

In [76]:
# 여러 개의 저장 파일을 통합하기
jeju_insta_df = pd.DataFrame([ ])

folder = './data/'
f_list = ['jejudo_matjip.xlsx', 'jeju_matjip.xlsx', 'jejudo_yeohaeng.xlsx', 'jeju_yeohaeng.xlsx']

for fname in f_list:
    fpath = folder + fname
    temp = pd.read_excel(fpath)
    jeju_insta_df = jeju_insta_df.append(temp)
    
jeju_insta_df.columns = ['', 'content', 'date', 'like', 'place', 'tags']

In [77]:
# 중복 데이터를 제거하고 저장하기
jeju_insta_df.drop_duplicates(subset = ['content'], inplace=True)
jeju_insta_df.to_excel('./data/jeju_crawling_raw.xlsx', index=False)